<a href="https://colab.research.google.com/github/Tymass/chess-player/blob/yolo-training/YOLO_chess_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
!pip install torch torchvision
!pip install ultralytics
!pip install -U --no-cache-dir gdown --pre
!pip install roboflow
!pip install pyyaml

## Imports

In [ ]:
from ultralytics import YOLO
import gdown
import os
import torch
import glob
from roboflow import Roboflow
import shutil
import yaml
import time
from google.colab import userdata
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## Functions definition

## Constants

In [ ]:
# Configuration parameters for machine learning model training.

EPOCHS = 600
# Specifies the total number of passes through the entire training dataset.
# A high number like 1000 indicates intensive training, aiming for deep model refinement, but requires monitoring to avoid overfitting.

PATIENCE = 20
# Determines the number of epochs with no improvement on a chosen metric before training is stopped early.
# This is a strategy to prevent overfitting by halting the training process if the model ceases to learn further.

BATCH_TRAINING = -1
# Specify batch size during YOLO training. -1 here means automatic way

LR0 = 0.0001
# The initial learning rate sets the step size at the start of training for adjusting the model weights.
# An appropriate value (neither too large nor too small) is crucial for effective training dynamics.

LRF = 0.2
# Final learning rate factor, used to adjust the learning rate over time.
# This parameter is part of a learning rate schedule that gradually reduces the learning rate, aiding in model convergence.

IMGSZ = 640
# The uniform size (height and width) to which all input images are resized before being fed into the model.
# Ensuring a consistent image size is essential for models to process batches of data efficiently.

CONF_THRESHOLD = 0.5
# Confidence threshold for predictions. Only predictions with a confidence score higher than this threshold will be considered.
# This helps in reducing false positives by filtering out predictions with low confidence.

IOU_THRESHOLD = 0.6
# Intersection Over Union (IOU) threshold for Non-Maximum Suppression (NMS).
# Determines how much overlap between bounding boxes is allowed. A higher threshold can reduce duplicate detections for the same object.

# Setting up device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Set the current working directory as the base directory.
#BASE_DIR = os.getcwd()

## Datasets downloads

In [ ]:
# In case we need to remove dataset folder
!rm -rf /content/datasets

We use [1](https://universe.roboflow.com/joseph-nelson/chess-pieces-new/dataset/24) dataset as baseline then fine tune model on our dataset [2](https://app.roboflow.com/tymek-byrwa-1p3fh/chesspiecesdetection-y9ljv/6).

Dont forget to get your own Roboflow API key.


In [ ]:
# Define the mode for dataset selection. Possible values: 'yolov9' or 'yolov8'.
mode = 'yolov8'
classes_nb = 12
API_key = userdata.get('API_key')

# We need to specify what dataset we use
fine_tunning = True

if fine_tunning:
  project_version = 6

  if mode == 'yolov9':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("tymek-byrwa-1p3fh").project("chesspiecesdetection-y9ljv")
    version = project.version(project_version)
    dataset = version.download("yolov9")
  elif mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("tymek-byrwa-1p3fh").project("chesspiecesdetection-y9ljv")
    version = project.version(project_version)
    dataset = version.download("yolov8")
else:
  project_version = 24

  if mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("joseph-nelson").project("chess-pieces-new")
    version = project.version(project_version)
    dataset = version.download("yolov8")
  elif mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("joseph-nelson").project("chess-pieces-new")
    version = project.version(project_version)
    dataset = version.download("yolov9")

In [ ]:
def update_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Modify the specific fields
    data['train'] = '../train/images'
    data['val'] = '../valid/images'

    with open(file_path, 'w') as file:
        yaml.safe_dump(data, file)

In [ ]:
# We move downloaded dataset to 'dataset' floder and update yaml file

os.mkdir('/content/datasets')
source_dataset_path = os.getcwd() + '/' + dataset.name.replace(' ', '-') + '-' + project_version
source_dataset_path_copy = source_dataset_path.split('/')[1:]

source_dataset_path_copy.insert(-1, 'datasets')


dest_dataset_path = '/' + '/'.join(source_dataset_path_copy)
config_path = dest_dataset_path + '/data.yaml'
test_imgs_path = dest_dataset_path + '/test/images'

shutil.move(source_dataset_path, dest_dataset_path)

update_yaml(config_path)

## Initial train

In [ ]:
# Check the selected mode and initialize the corresponding YOLO model.
if mode == 'yolov9':
    model = YOLO('yolov9e.pt')  # Initialize YOLOv9 with the specified weights file.
elif mode == 'yolov8':
    model = YOLO('yolov8m.pt')  # Initialize YOLOv8 with the specified weights file.
else:
    print(f"Set up config var mode correctly.")  # Prompt to set the mode variable correctly if it's not 'yolov9' or 'yolov8'.

# Train the model with specified parameters.
model.train(data=config_path,   # Path to the YAML file with dataset paths.
            save=True,             # Enable saving the final model weights.
            epochs=EPOCHS,         # Number of epochs to train the model.
            device=DEVICE,          # Device setup
            patience=PATIENCE,     # Early stopping patience.
            batch=BATCH_TRAINING,  # Batch size for training.
            imgsz=IMGSZ,           # Input image size.
            lr0=LR0,               # Initial learning rate.
            lrf=LRF)               # Final learning rate.

# Evaluate the model and compute the validation metrics.
metrics = model.val()

# Save the computed metrics to a text file.
with open(f'{mode}_base_model_metrics.txt', 'w') as f:
    f.write(f"mAP 50-95: {metrics.box.map}\n")   # Write mAP at IoU=50:95.
    f.write(f"mAP 50: {metrics.box.map50}\n")    # Write mAP at IoU=50.
    f.write(f"mAP 75: {metrics.box.map75}\n")    # Write mAP at IoU=75.
    f.write(f"Category-wise mAP 50-95: {metrics.box.maps}\n")  # Write list of mAP50-95 for each category.


## Inference of initial trained model

In [ ]:
model.predict(source=test_imgs_path,
                conf=CONF_THRESHOLD,
                iou=IOU_THRESHOLD,
                imgsz=IMGSZ,
                device=DEVICE,
                save=True)

In [ ]:
# Calculate predictions time
def predictions_time(path):
    try:
        if not os.path.isdir(path):
            print("Podana ścieżka nie istnieje lub nie prowadzi do folderu.")
            return
        files = os.listdir(path)

        for file in files:
            t1 = time.time()

            results = model.predict(path + '/' + file)

            t2 = time.time()
            dt = t2 - t1

            print(f"Detection time: {dt:.4f} seconds")


    except Exception as e:
        print("Wystąpił błąd:", e)

In [ ]:
predictions_time('/content/datasets/chessPiecesDetection-6/test/images')